In [159]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

[]
2.14.0


In [160]:
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
from sklearn.model_selection import train_test_split
import pandas as pd
# augmentasi gambar
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Menetapkan batas waktu pelatihan
import time
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image  # This should now work without errors.
# from google.colab import drive
# drive.mount('/content/drive')

In [161]:
# dari bapaknya
base_dir = 'C:/xampp/htdocs/dataset/animal_dataset'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
folders=os.listdir(train_dir)

#tambahan sendiri
image_paths = []
labels = []
for class_name in folders:
    class_dir = os.path.join(train_dir, class_name)
    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)
        image_paths.append(img_path)
        labels.append(class_name)

In [ ]:
# Split dataset into training (80%) and validation (20%)
# X_train, X_val, y_train, y_val = train_test_split(image_paths, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(image_paths, labels, test_size=0.3, random_state=42)
print(len(X_train),len(X_val))
print(len(y_train),len(y_val))

16 4
16 4


In [183]:
# train_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest',
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    )

val_datagen = ImageDataGenerator(rescale=1./255)
# dari bapaknya
# train_generator = train_datagen.flow_from_directory(
#                     train_dir,
#                     target_size=(150, 150),
#                     batch_size=1
#                     )
# validation_generator = val_datagen.flow_from_directory(
#                     validation_dir,
#                     target_size=(150, 150),
#                     batch_size=1
#                     )

# buatan sendiri
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': X_train, 'class': y_train}), # direktori data latih
    x_col='filename',
    y_col='class',
    target_size=(150,150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
    batch_size=1,
    class_mode='categorical', # menggunakan categorical karena klasifikasi lebih dari 2 kelas
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': X_val, 'class': y_val}), # direktori data validasi
    x_col='filename',
    y_col='class',
    target_size=(150,150), # mengubah resolusi gambar menjadi 150x150 piksel
    batch_size=1,
    class_mode='categorical',
)

Found 16 validated image filenames belonging to 5 classes.
Found 4 validated image filenames belonging to 3 classes.


In [165]:
from tensorflow.keras import layers
from tensorflow.keras import Model
# img_input = layers.Input(shape=(150, 150, 3))
# x = layers.Conv2D(16, 3, activation='relu')(img_input)
# x = layers.MaxPooling2D(2)(x)
# x = layers.Conv2D(32, 3, activation='relu')(x)
# x = layers.MaxPooling2D(2)(x)
# # start
# x = layers.Conv2D(64, 3, activation='relu')(x)
# x = layers.MaxPooling2D(2)(x)
# # x = layers.Conv2D(128, 3, activation='relu')(x)
# # x = layers.MaxPooling2D(2)(x)
# # end
# x = layers.Flatten()(x)
# # x = layers.Dense(50, activation='sigmoid')(x)
# x = layers.Dense(64, activation='relu')(x)
# Dropout(0.5),
# output = layers.Dense(5, activation='softmax')(x)
# model = Model(img_input, output)
# # model.compile(loss='mean_squared_error', optimizer='SGD', metrics=['acc'])
# model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
# model.summary()

# model CNN buatan sendiri
model = tf.keras.models.Sequential([
    Conv2D(32,(3,3),activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(512,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax') # 5 classes output
])
model.compile(optimizer='SGD',
    loss='mean_squared_error',
    metrics=['accuracy'])
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_54 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_54 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_55 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_55 (MaxPooli  (None, 36, 36, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_56 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_56 (MaxPooli  (None, 17, 17, 128)      

In [166]:
history = model.fit(
            train_generator,
            # steps_per_epoch=20,
            epochs=250,
            validation_data=validation_generator,
            # validation_steps=10,
            # verbose=2
            )

Epoch 1/250
15/15 [==============================] - ETA: 0s - loss: 0.1602 - accuracy: 0.2667

InvalidArgumentError: Graph execution error:

Detected at node mean_squared_error/SquaredDifference defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Python311\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "c:\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "c:\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "c:\Python311\Lib\asyncio\events.py", line 80, in _run

  File "c:\Python311\Lib\site-packages\ipykernel\kernelbase.py", line 516, in dispatch_queue

  File "c:\Python311\Lib\site-packages\ipykernel\kernelbase.py", line 505, in process_o

In [ ]:
# Load history
history = model.history

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, color='b', label='Train Accuracy')
plt.plot(epochs, val_acc, color='r', label='Validation Accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, color='b', label='Train Loss')
plt.plot(epochs, val_loss, color='r', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

# Simpan atau panggil kembali `model`

In [ ]:
# model.save('/home/irium/htdocs/widiarrohman1234.github.io/Artificial_Intelligence/program_application/UAS_AI_S2_Prof_Ali_Barakbah_Class_Animal/model_e1000_EarlyStopping.h5')

In [ ]:
# panggil model jika sudah pernah dibuat sebelumnya
# jika pertama kali menjalankan, komentar kode ini agar tidak error
# model = tf.keras.models.load_model('/home/irium/htdocs/widiarrohman1234.github.io/Artificial_Intelligence/program_application/UAS_AI_S2_Prof_Ali_Barakbah_Class_Animal/model_e1000.h5')  # Ganti dengan nama file model Anda


# Tahap ini untuk melakukan Prediksi

In [ ]:
# buatan sendiri
for filename in os.listdir(test_dir):
    # Make sure the file is an image (check for .jpg, .jpeg, .png, etc.)
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img = load_img(f"{test_dir}//{filename}", False, target_size=(150,150))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        preds = model.predict(x)
        print("Nilai Output Units:\n", preds)
        index_preds = np.argmax(preds)
        img_print = image.load_img(f"{test_dir}//{filename}", target_size=(150, 150))
        plt.imshow(img_print)
        plt.title(f'Predicted animal: {folders[index_preds]}')
        plt.show()
        print("############################################")

# Identitas Penulis
- Nama: Widi Arrohman
- Email: widiarrohman1234@gmail.com